In [1]:
import json
import os
import pickle
import sys
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
import networkx as nx
import torch
from frank.graph import InferenceGraph

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import utils
from src.traverse import Traverse

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct",
    # token='hf_xxx'
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    local_files_only=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.00s/it]


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

In [4]:
def create_subgraphs(T):

    dec = T.traverse('dec')
    agg = T.traverse('agg')
    full_graph = [*dec, *agg]

    alist_layers = []
    for layer in full_graph:
        for edges in layer.values():
            u = utils.format_alist(T.G.alist(edges[0][0]))
            data = edges[0][2]
            V = [utils.format_alist(T.G.alist(v)) for _, v, _ in edges]
            alist_layers.append((u, data, V))

    user_contents = []
    for u, data, V in alist_layers:
        user_contents.append(f"The node with data {u} is the current node. The data associated with the node is {data}. The node has the following children nodes: {V}. Explain the relationship between the current node and its children nodes.")

    return user_contents

In [5]:
def single_alists(T):

    alists = [T.G.alist(n).attributes for n in T.G.nodes]

    user_contents = []
    for alist in alists:
        user_contents.append(f"Explain the data associated with the current node {alist}.")

    return user_contents

In [6]:
def assemble_messages(user_content, system_content):

    inputs = []
    for u in user_content:

        messages = [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": u
            }
        ]

        inputs.append(messages)

    return inputs

In [7]:
def generate_response(inputs):

    input_messages = tokenizer.apply_chat_template(inputs, padding=True, tokenize=True, return_tensors="pt", add_generation_prompt=True).to(device)
    outputs = model.generate(input_messages, max_new_tokens=256)
    responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return responses

In [8]:
with open("../resources/frank_query_pickles/unsorted/2024-05-17T12:43:05.553797.pkl", 'rb') as f:
    G = pickle.load(f)['graph']

T = Traverse(G, remove_post_lookup=False)
dec = T.traverse('dec')
agg = T.traverse('agg')
full_graph = [*dec, *agg]

system_content = "Transform the following attribute-value structure into natural language text."

In [9]:
alists = single_alists(T)
inputs = assemble_messages([alists[3]], system_content)

In [10]:
responses = generate_response(inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 42.75 GiB. GPU 0 has a total capacty of 79.11 GiB of which 28.32 GiB is free. Process 863546 has 50.78 GiB memory in use. Of the allocated memory 47.97 GiB is allocated by PyTorch, and 2.15 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(responses[0])

system

Transform the following attribute-value structure into natural language text.user

Explain the data associated with the current node {'id': '0', 'h':'min','s': '?x3', 'p': 'urban population', 'o': '$y0', 'v': '$y0', 'u': 0.2493739240705097, 't': '2010', 'xp': '', 'fp': '', 'cx': '','meta': {'cost': 0.0, 'depth': 0,'state': 2, 'data_sources': ['worldbank', 'wikidata'], 'branch_type': 'or', 'node_type': 'znode'}, 'what': 'The entity whose urban population in 2010 has the minimum value of 47880.0 is Seychelles.', '?x3': 'Seychelles', 'how': '', 'why': 'Need to solve the sub-queries before determining the urban population in 2010.', '$y0': 47880.0}.assistant

Here is the natural language text associated with the current node:

"The entity whose urban population in 2010 has the minimum value of 47,880.0 is Seychelles."

This text summarizes the information stored in the node, including the entity (Seychelles), the attribute (urban population), the value (47,880.0), and the time peri